In [133]:
import requests
import pandas as pd
from splinter import Browser
from time import sleep
from bs4 import BeautifulSoup
from math import ceil
from difflib import SequenceMatcher

In [18]:
#takes a list_iterator object and pull the children from it returning a list
def children_to_list(list):
    result = []
    for l in list:
        result.append(l)
        
    return result # RETURNS LIST

In [175]:
#RETURNS LIST
def scrape_player_data(player):
    name = test[1][0:5] + test[0][0:2]
    number = 0
    url = f'https://www.baseball-reference.com/players/{name.lower()[0]}/{name.lower()}0{number}.shtml'
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    if '404' in soup.find_all('h1'):
        number += 1
        url = f'https://www.baseball-reference.com/players/{name.lower()[0]}/{name.lower()}0{number}.shtml'
        r = requests.get(url)
    
    stats = []   
    
    child_list = children_to_list(soup.find_all('div', class_ = 'stats_pullout')[0].children)
    
    if len(child_list[0].find_all('p')) == 2:
        for child in child_list:
            p_list = child.find_all('p')
            for i in range(len(p_list)):
                if (i % 2 == 1):
                    stats.append(p_list[i].text)
    elif len(child_list[0].find_all('p')) == 1:
        for child in child_list:
            p_list = child.find_all('p')
            for p in p_list:
                stats.append[p.text]
    return stats

In [3]:
player_df = pd.read_csv('nfl_av_ratings/nfl_av_ratings_all.csv')

In [12]:
player_df = player_df[['Rk', 'Player', 'From', 'To', 'Draft', 'Tm', 'Lg', 'G','GS', 'Yrs', 'PB', 'AP1', 'AV']]
player_df.head()

,Rk,Player,From,To,Draft,Tm,Lg,G,GS,Yrs,PB,AP1,AV
0,1,Tom Brady\BradTo00,2009,2018,6-199,NWE,NFL,156.0,156.0,10,10,2,165
1,2,Drew Brees\BreeDr00,2009,2018,2-32,NOR,NFL,157.0,157.0,10,9,0,161
2,3,Aaron Rodgers\RodgAa00,2009,2018,1-24,GNB,NFL,142.0,142.0,10,7,2,155
3,4,Matt Ryan\RyanMa00,2009,2018,1-3,ATL,NFL,158.0,158.0,10,4,1,152
4,5,Philip Rivers\RivePh00,2009,2018,1-4,SDG,NFL,160.0,160.0,10,7,0,143


In [17]:
player_list = player_df['Player']
player_list[0].split('\\')[1]

'BradTo00'

In [31]:
name = player_list[0].split('\\')[1]
url = f'https://www.pro-football-reference.com/players/{name[0]}/{name}.htm'
url

'https://www.pro-football-reference.com/players/B/BradTo00.htm'

In [21]:
r = requests.get(url)

In [23]:
soup = BeautifulSoup(r.text, 'html.parser')

In [41]:
meta = soup.find('div', class_ = 'players').find('div').find_all('p')

In [59]:
meta[5].find_all('p')

[<p><strong><a href="/blog/?page_id=518">Weighted Career AV (100-95-...)</a>:</strong> 175 (2nd overall since 1960)
 </p>, <p><strong>High School</strong>: 
   
 		<a href="/schools/high_schools.cgi?id=93b8d6c3">Junipero Serra</a> (<a href="/schools/high_schools.cgi?hs_state=CA">CA</a>)
 	
 </p>, <p><strong>Draft</strong>: <a href="/teams/nwe/2000_draft.htm">New England Patriots</a> in the 6th round (199th overall) of the <a href="/years/2000/draft.htm">2000 NFL Draft</a>.</p>, <p><strong>Current salary</strong>: <a href="/players/salary.htm">14,000,000</a></p>]

In [65]:
r = requests.get('https://api.nfl.com/v1/persons/32004d41-4371-0352-e57d-251eacb121bd?fs={playerStats{teamStats{passing}}}')

In [66]:
r.status_code

401

In [71]:
states = ["Alaska","Alabama","Arkansas","American Samoa","Arizona","California","Colorado","Connecticut","District of Columbia","Delaware","Florida","Georgia","Guam","Hawaii","Iowa","Idaho","Illinois","Indiana","Kansas","Kentucky","Louisiana","Massachusetts","Maryland","Maine","Michigan","Minnesota","Missouri","Mississippi","Montana","North Carolina","North Dakota","Nebraska","New Hampshire","New Jersey","New Mexico", "Nevada", "New York","Ohio","Oklahoma","Oregon","Pennsylvania", "Rhode Island", "South Carolina","South Dakota","Tennessee","Texas","Utah","Virginia","Virgin Islands","Vermont","Washington","Wisconsin","West Virginia","Wyoming"]

In [84]:
url = f'https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_{states[1]}'

In [85]:
r = requests.get(url)

In [86]:
soup = BeautifulSoup(r.text, 'html.parser')

In [121]:
u_list = soup.find(class_ = 'mw-parser-output').find_all('th')[7::]

In [122]:
for i in range(len(u_list)):
    if '[b]' in u_list[i].text:
        u_list[i] = u_list[i].text.strip()
        u_list[i] = u_list[i][0:len(u_list[i])-3]
    else:
        u_list[i] = u_list[i].text.strip()
        

In [126]:
df = pd.DataFrame(columns = ['state', 'school'], index = [i for i in range(len(u_list))])
df['state'] = states[1]
df['school'] = u_list

In [128]:
df.to_csv('alabama_universities.csv')

In [134]:
u_df = pd.read_csv('Colleges_and_Universities.csv')

In [137]:
u_df = u_df[['NAME', 'STATE']]
len(u_df)

7520

In [138]:
df = pd.read_json('nfl_profiles.json')

In [139]:
df

,birth_date,birth_place,college,current_salary,current_team,death_date,draft_position,draft_round,draft_team,draft_year,height,high_school,hof_induction_year,name,player_id,position,weight
0,1967-05-12,"Bay City, TX",Baylor,None,None,None,34.0,2.0,Seattle Seahawks,1990.0,6-0,"Van Vleck, TX",NaN,Robert Blackmon,1809,DB,208.0
1,1970-07-20,"Louisville, KY",Kentucky,None,None,None,85.0,4.0,Seattle Seahawks,1993.0,6-3,"Holy Cross, KY",NaN,Dean Wells,23586,LB,248.0
2,1990-08-14,"Newton, MA",Oregon,"1,075,000",Miami Dolphins,None,46.0,2.0,Buffalo Bills,2013.0,6-3,"Los Gatos, CA",NaN,Kiko Alonso,355,ILB,238.0
3,1948-04-22,"Dallas, TX",North Texas,None,None,1999-10-15,126.0,5.0,New Orleans Saints,1970.0,6-2,"W.W. Samuell, TX",NaN,Steve Ramsey,18182,QB,210.0
4,1988-02-27,"Neptune, NJ",Miami (FL),None,None,None,NaN,NaN,None,NaN,6-0,"Neptune, NJ",NaN,Cory Nelms,16250,CB,195.0
5,1982-08-18,"Columbus, GA",Notre Dame,None,None,None,144.0,5.0,St. Louis Rams,2005.0,6-4,"Wheaton-Warrenville South, IL",NaN,Jerome Collins,4310,TE,267.0
6,1992-10-27,"Cincinnati, OH",Louisville,"1,762,000",Buffalo Bills,None,73.0,3.0,Buffalo Bills,2014.0,6-1,"Northwest, OH",NaN,Preston Brown,2701,ILB,251.0
7,1945-03-17,"Steubenville, OH",Wyoming,None,None,None,NaN,NaN,None,NaN,5-11,"Steubenville, OH",NaN,Hub Lindsey,13379,RB,196.0
8,1978-11-11,"Suitland, MD",Maryland,None,None,None,49.0,2.0,New York Jets,2001.0,5-10,"Suitland, MD",NaN,LaMont Jordan,11755,RB,230.0
9,1921-10-06,"Brooklyn, NY",Dartmouth,None,None,None,178.0,18.0,Pittsburgh Steelers,1945.0,5-11,"Boys, NY",NaN,Alex Wizbicki,24550,DB-HB,188.0
